<a href="https://colab.research.google.com/github/Hannah1011/2024-DA-Project_MZ/blob/main/traffic_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. Drive랑 연동

In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


## 2. 데이터 불러오기

In [7]:
# 필요한 라이브러리 import
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
## 엑셀 파일 불러오기
traffic_accidents = pd.read_excel('/content/drive/MyDrive/seoul_data/노인교통사고.xlsx')
crosswalks = pd.read_excel('/content/drive/MyDrive/seoul_data/구_동별 횡단보도.xlsx')

# 구_동 열 기준으로 병합
merged_data = pd.merge(traffic_accidents, crosswalks, on='구_동', how ='outer')

# 병합 결과 확인
print(merged_data.head())

           사고번호       구_동      년도    월     일  사고시간       사고일시      년월  사망자수  \
0  2.021010e+15  성북구 하월곡동  2021.0  1.0   2.0  11.0 2021-01-02  2101.0   0.0   
1  2.021011e+15  성북구 하월곡동  2021.0  1.0   5.0  10.0 2021-01-05  2101.0   0.0   
2  2.021012e+15  성북구 하월곡동  2021.0  1.0  19.0  14.0 2021-01-19  2101.0   0.0   
3  2.021041e+15  성북구 하월곡동  2021.0  4.0  13.0   6.0 2021-04-13  2104.0   0.0   
4  2.021042e+15  성북구 하월곡동  2021.0  4.0  19.0  18.0 2021-04-19  2104.0   0.0   

   중상자수  ...  공사형태1  공사형태2  공사형태3  공사형태4  공사형태5  공사형태6  공사형태8  공사형태9  공사형태10  \
0   1.0  ...  105.0    0.0    5.0   91.0  360.0    0.0    0.0    0.0    48.0   
1   1.0  ...  105.0    0.0    5.0   91.0  360.0    0.0    0.0    0.0    48.0   
2   0.0  ...  105.0    0.0    5.0   91.0  360.0    0.0    0.0    0.0    48.0   
3   1.0  ...  105.0    0.0    5.0   91.0  360.0    0.0    0.0    0.0    48.0   
4   0.0  ...  105.0    0.0    5.0   91.0  360.0    0.0    0.0    0.0    48.0   

   공사형태11  
0    34.0  
1    34.0  
2    34.

In [5]:
# 병합된 데이터셋을 엑셀파일로 저장
merged_data.to_excel('/content/drive/MyDrive/seoul_data/traffic_merged.xlsx', index=False)

In [6]:
traffic_merged = pd.read_excel('/content/drive/MyDrive/seoul_data/traffic_merged.xlsx')
traffic_merged.head()

,사고번호,구_동,년도,월,일,사고시간,사고일시,년월,사망자수,중상자수,...,공사형태1,공사형태2,공사형태3,공사형태4,공사형태5,공사형태6,공사형태8,공사형태9,공사형태10,공사형태11
0,2.021010e+15,성북구 하월곡동,2021.0,1.0,2.0,11.0,2021-01-02,2101.0,0.0,1.0,...,105.0,0.0,5.0,91.0,360.0,0.0,0.0,0.0,48.0,34.0
1,2.021011e+15,성북구 하월곡동,2021.0,1.0,5.0,10.0,2021-01-05,2101.0,0.0,1.0,...,105.0,0.0,5.0,91.0,360.0,0.0,0.0,0.0,48.0,34.0
2,2.021012e+15,성북구 하월곡동,2021.0,1.0,19.0,14.0,2021-01-19,2101.0,0.0,0.0,...,105.0,0.0,5.0,91.0,360.0,0.0,0.0,0.0,48.0,34.0
3,2.021041e+15,성북구 하월곡동,2021.0,4.0,13.0,6.0,2021-04-13,2104.0,0.0,1.0,...,105.0,0.0,5.0,91.0,360.0,0.0,0.0,0.0,48.0,34.0
4,2.021042e+15,성북구 하월곡동,2021.0,4.0,19.0,18.0,2021-04-19,2104.0,0.0,0.0,...,105.0,0.0,5.0,91.0,360.0,0.0,0.0,0.0,48.0,34.0


## 3. Min-Max 정규화
- 변수들의 범위가 매우 다를 때, 특히 모든 값이 양수일 때 min-max 정규화를 사용하면 각 변수를 동일한 범위로 맞출 수 있어서 모델 학습에 더 적합해진다.
`예) 사상자수는 1 ~ 11사이의 값인데, 횡단보도개수는 1 ~ 2139개와 같이 큰 차이가 있기에 각 변수를 0~1 사이로 변환`

In [ ]:
# 정규화할 수치형 변수 목록
columns_to_normalize = ['사망자수', '중상자수', '경상자수','부상신고자수','사상자수', '피해운전자 연령', '횡단보도_개수', '상태_양호', '상태_파손',
                        '상태_도색', '상태_노후', '횡단보도1', '횡단보도2', '횡단보도3',
                        '횡단보도4', '횡단보도5', '횡단보도6', '횡단보도7', '화면표시_안보기',
                        '화면표시_보기', '공사형태1', '공사형태2', '공사형태3', '공사형태4',
                        '공사형태5', '공사형태6', '공사형태8', '공사형태9', '공사형태10',
                        '공사형태11']

#Min-Max 스케일러 생성
scaler = MinMaxScaler()

# 정규화된 열 이름에 '_norm'을 추가
for col in columns_to_normalize:
    traffic_merged[col + '_norm'] = scaler.fit_transform(traffic_merged[[col]])

# 정규화된 데이터 확인
print(traffic_merged[[col + '_norm' for col in columns_to_normalize]].head())


In [ ]:
# 정규화된 데이터를 엑셀 파일로 저장 (선택 사항)
traffic_merged.to_excel('normalized_traffic_merged_with_original.xlsx', index=False)